In [24]:
"""This function takes in a local json filename and reads it into a single list. It also removes punctuation and null values.
It has default punctuation it keeps "(, ), -, ", but this can be added to.""" 

#TO-DO:
#remove string if only numbers
#document this better, add doc_strings to methods and overall just make better 


import Ruk_Reader
import json
from nltk.tokenize import RegexpTokenizer
import string
import re
from nltk.corpus import stopwords

def json_cleaner(local_book_title, extra_strings_to_keep = "", print_state = True ):
    
    #this method removes the noise from array
    def _remove_noise(input_text, noise_list):
        if isinstance(input_text, list):
            for word in input_text:
                letters = list(word)
                for letter in letters:
                    if letter in noise_list:
                        letters.remove(letter)
                word = "".join(letters)
                noise_free_words.append(word) 
            return noise_free_words.split()
    
        elif isinstance(input_text, str):
            letters = list(input_text)
            for letter in letters:
                if letter in noise_list:
                    letters.remove(letter)
            noise_free_word = "".join(letters)
            return noise_free_word
    

#Deciding the noise/punctuation to remove from input list
    initial_noise = ["''"] 
    updated_punct = ''.join(i for i in string.punctuation if not i in ['(', ')', '-']) #+ extra_strings_to_keep
    for i in range (0, len(updated_punct)):
        initial_noise.append(updated_punct[i])
    if extra_strings_to_keep:
        initial_noise.remove(extra_strings_to_keep)
        
    
#instantiating lists
    all_words = []
    stripped = []
    stop_words = set(stopwords.words('english'))


#reading file
    with open(local_book_title, "r") as read_file:
        data = json.load(read_file)
    
    print("data is ", type(data))

#looks at keys of dictionary that is the data read. The main content is in the "chap-content"
    print(type(data))
    if isinstance(data, dict):
        subchap_headings = data.keys()
        print("dictionary keys: ", subchap_headings)  #these are probably removable
        print()
        all_content = data.get("chap-content") 
    elif isinstance(data, str):
        all_content = data.split()
    elif isinstance(data, list):
        all_content = data

#Turns list of lists into single list, also removes stopwords, and makes lowercase

    for i in range(0, len(all_content)):
        #all_content[i] = _remove_noise(all_content[i], initial_noise)
        #all_content[i] = all_content[i].split()
        if all_content[i]:
            all_content[i] = _remove_noise(all_content[i], initial_noise)
            all_content[i] = all_content[i].split()
            for j in range(0, len(all_content[i])):
                if all_content[i][j] and "http" not in all_content[i][j] and all_content[i][j] not in stop_words and all_content[i][j].isdigit() == False:
                    all_words.append(all_content[i][j].lower())
    
    if print_state:
        print(all_words)
    
    else:
        return all_words
    

In [116]:
"""This function takes in a bunch of things and creates a word2vec CBOW NN"""

import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import spacy  # For preprocessing
from spacy.lang.en import English  #not sure what this is for, check if needed
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

#word2vec stuff here
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser  #for bigrams/common phrase ID
#import json_read_clean as jRC 

#TO-DO:
#2) document
#3) Test various parameters for word2vec functions
#4) maybe write it such that it can take in a list of lists, each list being all the words in a book
#5) TF-IDF
#6) Maybe make it return a word2vec model that then can be probed? for instance to get cosine similarity?


def word2vec_book_model(list_of_book_words, min_count=2, window = 2, size = 300, sample = 6e-5, alpha=0.03
                       ,min_alpha = 0.0007, negative = 0, workers = 6):

    phrases = Phrases(list_of_book_words, min_count=30, progress_per=10000)   #also sent is a list here 
    bigram = Phraser(phrases)
    sentences = bigram[list_of_book_words]
    
    cores = multiprocessing.cpu_count() # Count the number of cores in a computer, it's 8

#1) INITALIZING THE MODEL
    w2v_model = Word2Vec(min_count = min_count,         #Ignores all words with total absolute frequency lower than this - (2, 100)
                     window = window,           #The maximum distance between the current and predicted word within a sentence.
                     size = size,           #size of feature vectors
                     sample = sample,   #threshold for configuring which higher-frequency words are randomly downsampled. influencial. (0, 1e-5)
                     alpha = alpha,        #learning rate initial (0.01 - 0.05)
                     min_alpha = min_alpha,     #final learning rate, linearly drops to this as training goes
                     negative = negative,  # If > 0,  how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)
                     workers = workers)        #how many cores doing the work

#2) BUILDING VOCAB
    t = time()
    w2v_model.build_vocab(sentences, progress_per=10000)
    print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
    
#3) TRAINING MODEL

    #    total_examples = int, Count of sentences;
     #   epochs = int - Number of iterations (epochs) over the corpus - [10, 20, 30]
        #total_examples=w2v_model.corpus_count
    
    t = time()
    w2v_model.train(sentences, total_examples = len(sentences), epochs=30, report_delay=1)
    print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))


    w2v_model.init_sims(replace=True)  #do this when done training the model? normalizes the length of the vectors before comparing
    
    print("the size of the vocabulary is: ", len(w2v_model.wv.vocab))
#    print("The words most similar to polymers are: ", w2v_model.wv.most_similar(positive=["polymer"]))
#    print("cosine similarity between polymer and plastic: ", w2v_model.wv.similarity("polymer", "polymerization"))
    print("size of the corpus_count:", w2v_model.corpus_count)  #
    return w2v_model

In [43]:
Ravve = json_cleaner("Principles_of_Polymer_Chemistry_by_Ravve.json", "", False)
Schaller = json_cleaner("Schaller_polym_chem", "", False)
Soderberg = json_cleaner("Soderberg_bio_o_chem", "", False)
#print(Ravve_words)
Multiple_book_list = []
local_titles = ["Principles_of_Polymer_Chemistry_by_Ravve.json", "Schaller_polym_chem", "Soderberg_bio_o_chem"]
    
for title in local_titles:
    list_of_words = json_cleaner(title, "", False)
    Multiple_book_list.append(list_of_words)
    print(len(Multiple_book_list))
    print("length of ", title, "is ", len(list_of_words))
    
for book in Multiple_book_list:
    print(len(book))

#print((Multiple_book_list[:1000]))

#"Schaller_polym_chem"   Principles_of_Polymer_Chemistry_by_Ravve.json

data is  <class 'str'>
<class 'str'>
data is  <class 'dict'>
<class 'dict'>
dictionary keys:  dict_keys(['chap-title', 'chap-content'])

data is  <class 'list'>
<class 'list'>
data is  <class 'str'>
<class 'str'>
1
length of  Principles_of_Polymer_Chemistry_by_Ravve.json is  187534
data is  <class 'dict'>
<class 'dict'>
dictionary keys:  dict_keys(['chap-title', 'chap-content'])

2
length of  Schaller_polym_chem is  10082
data is  <class 'list'>
<class 'list'>
3
length of  Soderberg_bio_o_chem is  91495
187534
10082
91495


In [216]:
#word2vec_book_model(Multiple_book_list, min_count = 5, window = 3)

three_book_model = word2vec_book_model(Multiple_book_list, min_count = 3, window = 4, size = 300, sample = 6e-5)


INFO - 21:27:40: collecting all words and their counts
INFO - 21:27:40: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 21:27:42: collected 218149 word types from a corpus of 289111 words (unigram + bigrams) and 3 sentences
INFO - 21:27:42: using 218149 counts as vocab in Phrases<0 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 21:27:42: source_vocab length 218149
INFO - 21:27:46: Phraser built with 204 phrasegrams
INFO - 21:27:46: collecting all words and their counts
INFO - 21:27:47: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 21:27:48: collected 30016 word types from a corpus of 273282 raw words and 3 sentences
INFO - 21:27:48: Loading a fresh vocabulary
INFO - 21:27:48: effective_min_count=3 retains 9650 unique words (32% of original 30016, drops 20366)
INFO - 21:27:48: effective_min_count=3 leaves 248885 word corpus (91% of original 273282, drops 24397)
INFO - 21:27:48: deleting the raw counts dictionary of 300

Time to build vocab: 0.04 mins


INFO - 21:27:48: training model with 6 workers on 9650 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=0 window=4
INFO - 21:27:50: EPOCH 1 - PROGRESS: at 0.00% examples, 0 words/s, in_qsize 0, out_qsize 0
INFO - 21:27:50: worker thread finished; awaiting finish of 5 more threads
INFO - 21:27:50: worker thread finished; awaiting finish of 4 more threads
INFO - 21:27:50: worker thread finished; awaiting finish of 3 more threads
INFO - 21:27:50: worker thread finished; awaiting finish of 2 more threads
INFO - 21:27:50: worker thread finished; awaiting finish of 1 more threads
INFO - 21:27:50: worker thread finished; awaiting finish of 0 more threads
INFO - 21:27:50: EPOCH - 1 : training on 273282 raw words (25854 effective words) took 1.6s, 16483 effective words/s
INFO - 21:27:51: EPOCH 2 - PROGRESS: at 0.00% examples, 0 words/s, in_qsize 0, out_qsize 0
INFO - 21:27:52: worker thread finished; awaiting finish of 5 more threads
INFO - 21:27:52: worker thread finished; aw

Time to train the model: 0.83 mins


INFO - 21:28:38: precomputing L2-norms of word weight vectors


the size of the vocabulary is:  9650
size of the corpus_count: 3


In [211]:
#write a quick method here for simplifying the below

def model_analyzer(word2vec_model, word1_compare, word2_compare, similarity_word = ""):
    #print("similarity between alchohol and hydroxyl is: ", new_trained_model.wv.similarity("alcohol", "hydroxyl"))
    compare_num =  (word2vec_model.wv.similarity(word1_compare, word2_compare))
    print("similarity between", word1_compare, " and ", word2_compare, " is  ", word2vec_model.wv.similarity(word1_compare, word2_compare))

     
    if similarity_word != "":
        print("words most similar to", similarity_word, "are: ")
        print(word2vec_model.wv.most_similar(positive=[similarity_word], topn = 10))

    return compare_num


model_analyzer(three_book_model, "alcohol", "hydroxyl", similarity_word = "polymer")

similarity between alcohol  and  hydroxyl  is   0.9994086
words most similar to polymer are: 
[('linear', 0.9997829794883728), ('crystallinity', 0.999779999256134), ('molten', 0.9997320175170898), ('polymeric_chains', 0.9997013807296753), ('crystalline', 0.9996921420097351), ('state', 0.9996786713600159), ('related', 0.9996733665466309), ('exhibit', 0.9996394515037537), ('speciﬁc', 0.9996343851089478), ('increases', 0.9996246099472046)]


0.9994086

In [217]:
def syn_ant_tester(nn_model):

    elec_nuc = model_analyzer(nn_model, "electrophile", "nucleophile")   #opposites
    acid_base = model_analyzer(nn_model,"acid", "base")      #opposites
    polar_nonpol = model_analyzer(nn_model,"polar", "nonpolar")   #opposites
    synth_nat = model_analyzer(nn_model,"synthetic", "natural")   #opposites
    pos_neg = model_analyzer(nn_model,"positive", "negative")   #opposites
    #fs = model_analyzer(nn_model,"fast", "slow")   #opposites
    #db = model_analyzer(nn_model,"dark", "bright")   #opposites
    sc = model_analyzer(nn_model,"simple", "complex")   #opposites
    #mc = model_analyzer(nn_model,"polyamides", "ligand")   #opposites

    ant_avg = (elec_nuc+acid_base+polar_nonpol+synth_nat+pos_neg+sc)/7
    print("ant avg: ", ant_avg)
    print()
    #print(type(nn_model.wv.vocab))
    cn = model_analyzer(nn_model,"carbon", "nitrogen")  #related- same ish
    kc = model_analyzer(nn_model,"ketone", "carbonyl") #same ish 
    ah = model_analyzer(nn_model,"alcohol", "hydroxyl")  #same
    ph = model_analyzer(nn_model,"proton", "hydrogen")  #same  
    pc = model_analyzer(nn_model,"polymer", "chain")  #same
    en = model_analyzer(nn_model,"electron", "negative")  #same
    #hw = model_analyzer(nn_model,"water", "h2o")  #same
    

    syn_avg = (cn+kc+ah+ph+pc+en)/6
    print("syn avg: ", syn_avg)


syn_ant_tester(three_book_model)



similarity between electrophile  and  nucleophile  is   -0.011528181
similarity between acid  and  base  is   -0.01263427
similarity between polar  and  nonpolar  is   -0.05086182
similarity between synthetic  and  natural  is   -0.08032474
similarity between positive  and  negative  is   -0.025542585
similarity between simple  and  complex  is   -0.049809996
ant avg:  -0.032957370792116435

similarity between carbon  and  nitrogen  is   -0.012445969
similarity between ketone  and  carbonyl  is   0.032217734
similarity between alcohol  and  hydroxyl  is   0.0010377867
similarity between proton  and  hydrogen  is   0.04891874
similarity between polymer  and  chain  is   0.033405773
similarity between electron  and  negative  is   -0.030572325
syn avg:  0.012093623479207357


In [206]:
Holder = []
Holder.append(Ravve)
Holder.append(Soderberg)
Ravve_Soder = word2vec_book_model(Holder, min_count = 3, window = 4, size = 300, sample = 6e-5, negative = 10)
#syn_ant_tester(Ravve_Model)


INFO - 14:16:38: collecting all words and their counts
INFO - 14:16:38: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 14:16:39: collected 210593 word types from a corpus of 279029 words (unigram + bigrams) and 2 sentences
INFO - 14:16:39: using 210593 counts as vocab in Phrases<0 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 14:16:39: source_vocab length 210593
INFO - 14:16:43: Phraser built with 200 phrasegrams
INFO - 14:16:43: collecting all words and their counts
INFO - 14:16:44: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 14:16:44: collected 29497 word types from a corpus of 263489 raw words and 2 sentences
INFO - 14:16:44: Loading a fresh vocabulary
INFO - 14:16:44: effective_min_count=3 retains 9447 unique words (32% of original 29497, drops 20050)
INFO - 14:16:44: effective_min_count=3 leaves 239494 word corpus (90% of original 263489, drops 23995)
INFO - 14:16:45: deleting the raw counts dictionary of 294

Time to build vocab: 0.03 mins


INFO - 14:16:45: training model with 6 workers on 9447 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=10 window=4
INFO - 14:16:46: worker thread finished; awaiting finish of 5 more threads
INFO - 14:16:46: worker thread finished; awaiting finish of 4 more threads
INFO - 14:16:46: worker thread finished; awaiting finish of 3 more threads
INFO - 14:16:46: worker thread finished; awaiting finish of 2 more threads
INFO - 14:16:46: EPOCH 1 - PROGRESS: at 50.00% examples, 6598 words/s, in_qsize 1, out_qsize 1
INFO - 14:16:46: worker thread finished; awaiting finish of 1 more threads
INFO - 14:16:46: worker thread finished; awaiting finish of 0 more threads
INFO - 14:16:46: EPOCH - 1 : training on 263489 raw words (20000 effective words) took 1.5s, 13088 effective words/s
INFO - 14:16:48: worker thread finished; awaiting finish of 5 more threads
INFO - 14:16:48: worker thread finished; awaiting finish of 4 more threads
INFO - 14:16:48: worker thread finished; awaiting fini

Time to train the model: 0.79 mins


INFO - 14:17:33: precomputing L2-norms of word weight vectors


the size of the vocabulary is:  9447
size of the corpus_count: 2


In [208]:
syn_ant_tester(Ravve_Soder)
#print(Schaller_Model.wv.vocab)

similarity between electrophile  and  nucleophile  is   0.007978465
similarity between acid  and  base  is   0.99992704
similarity between polar  and  nonpolar  is   0.9999286
similarity between synthetic  and  natural  is   0.99992424
similarity between positive  and  negative  is   0.999778
similarity between fast  and  slow  is   0.0822365
similarity between simple  and  complex  is   0.9999374
similarity between polyamides  and  ligand  is   0.99967813
ant avg:  0.7611734867095947

similarity between carbon  and  nitrogen  is   0.9999639
similarity between ketone  and  carbonyl  is   0.9999128
similarity between alcohol  and  hydroxyl  is   0.9996463
similarity between proton  and  hydrogen  is   0.9987124
similarity between polymer  and  chain  is   0.99991715
similarity between electron  and  negative  is   0.99963546
syn avg:  0.999631404876709
